In [9]:
% matplotlib inline
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import networkx as nx
import nltk
from langdetect import detect
from langdetect import detect_langs
from langdetect import DetectorFactory
from nltk.tokenize import RegexpTokenizer  
from stop_words import get_stop_words  
from nltk.stem.porter import PorterStemmer  
from gensim import corpora, models  
import gensim  



In [10]:
mongo_username = 'Campione'
mongo_password = 'veTRxJL29lpKWwPn'
mongo_url = 'mongodb://%s:%s@cluster0-shard-00-01-i6gcp.mongodb.net:27017/admin' % (
            mongo_username, mongo_password)
client = MongoClient(mongo_url,ssl=True,replicaSet='Cluster0-shard-0',authSource='admin')
info = client.server_info()  # Forces a call.
print(info)
print(client.database_names())

{'version': '3.4.14', 'gitVersion': 'fd954412dfc10e4d1e3e2dd4fac040f8b476b268', 'modules': ['enterprise'], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [3, 4, 14, 0], 'openssl': {'running': 'OpenSSL 1.0.1e-fips 11 Feb 2013', 'compiled': 'OpenSSL 1.0.1e-fips 11 Feb 2013'}, 'buildEnvironment': {'distmod': 'rhel70', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v2/bin/gcc: gcc (GCC) 5.4.0', 'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp', 'cxx': '/opt/mongodbtoolchain/v2/bin/g++: g++ (GCC) 5.4.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -std=c++11', 'linkflags': '-pthread -Wl,-z,now -rdynamic -Wl,--fatal-warnings -fstack-protector-strong -fuse-ld=gold -Wl,-

In [11]:
db = client.gitdbPro
repos = db.repos
descriptions = repos.distinct('description')
df = pd.DataFrame(descriptions)
df.head()

,0
0,How to be low-level programmer
1,Import OpenStreetMap data into Unreal Engine 4
2,react-native template to target multiple platf...
3,Connect your App to Multiple Messaging Channel...
4,Proto Actor - Ultra fast distributed actors fo...


In [12]:
df[0]

0                           How to be low-level programmer
1           Import OpenStreetMap data into Unreal Engine 4
2        react-native template to target multiple platf...
3        Connect your App to Multiple Messaging Channel...
4        Proto Actor - Ultra fast distributed actors fo...
5                用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
6                                   pagehelper-spring-boot
7            📅 A date picker components for ionic2 ／ionic3
8        Track prices on Amazon and receive email alert...
9        Advanced web based trading client for the Stel...
10                                  Join our slack channel
11       An Elm-based prototype to help designers build...
12       Can neural networks transliterate Romaji into ...
13                              A repo with small projects
14               Go - CQRS / Event Sourcing made easy - Go
15                                RNN Tutorial for Artists
16       A macOS app to convert JSON objects into Swift.

In [19]:
DetectorFactory.seed = 0

'zh-cn'

In [20]:
cn_text_list = []
en_text_list = []
for item in df[0]:
    try:
        if detect(str(item)) == 'zh-cn':
            cn_text_list.append(item)
        else:
            en_text_list.append(item)
    except Exception as e:
        print(str(e))
        

No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.


In [23]:
len(cn_text_list)

697

In [24]:
len(en_text_list)

11143

In [28]:
df_cn = pd.DataFrame({'text':cn_text_list})
print(df_cn)
df_cn.to_csv('cn_text')

                                                  text
0            用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
1             每周一篇，内容精简，不咸不淡，期盼探讨。微信公众号：芋道源码【纯源码分享公众号】
2                                    iOS即时通讯，从入门到“放弃”？
3    『假装看天气』─── 天气预报 & 公交查询 & 资讯阅读 & 妹纸福利 的 Android...
4                                           微信小程序－日历 📅
5                                             一个表白用的页面
6                                     适用于微信小程序的图片预加载组件
7                                             尚妆产品技术刊读
8                   一个超级迷你轻量级全方向完美侧滑控件，兼容各种嵌套与被嵌套滑动控件。
9    这个是我个人网站的项目，欢迎贡献代码，力求能够应用到实际工作中java相关的大多数技术栈。有...
10                                       知乎爬虫（验证码自动识别）
11                                    微信小程序中的股票分时图、K线图
12                                        微信小程序开源项目库汇总
13             :dog: wux - 微信小程序自定义组件（对话框、指示器、五星评分...）
14                                        一个灵活、友好的爬虫框架
15   妈妈说，问人问题要先说你好，那么咱这个全平台的理论上做的最牛逼的对话框（只是理论上）就叫Le...
16                                            大鸡排的博客归档
17        

In [29]:
df_en = pd.DataFrame({'text':en_text_list})
print(df_en.head())
df_en.to_csv('en_text')

                                                text
0                     How to be low-level programmer
1     Import OpenStreetMap data into Unreal Engine 4
2  react-native template to target multiple platf...
3  Connect your App to Multiple Messaging Channel...
4  Proto Actor - Ultra fast distributed actors fo...


In [33]:
# GENERATE LDA Model

# list for tokenized documents in loop  
texts = []
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer  
p_stemmer = PorterStemmer()

# loop through document list  
for i in en_text_list:
    if i is not None:
        raw = i.lower()
        # clean and tokenize document string
        tokens = tokenizer.tokenize(raw)
        
        # remove stop words from tokens  
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        
        # add tokens to list
        texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
      
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=100, id2word = dictionary, passes=20)

/Users/haki/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [35]:
for topic in ldamodel.print_topics(num_words=5, num_topics=100):
    print(topic)

(0, '0.204*"time" + 0.106*"via" + 0.100*"real" + 0.047*"cnn" + 0.039*"use"')
(1, '0.239*"packag" + 0.059*"remot" + 0.034*"peopl" + 0.031*"pwa" + 0.024*"o"')
(2, '0.347*"project" + 0.110*"provid" + 0.054*"sdk" + 0.047*"base" + 0.043*"clean"')
(3, '0.182*"test" + 0.152*"visual" + 0.061*"structur" + 0.055*"dataset" + 0.035*"use"')
(4, '0.205*"user" + 0.089*"password" + 0.067*"domain" + 0.063*"authent" + 0.058*"check"')
(5, '0.163*"process" + 0.117*"aw" + 0.074*"will" + 0.065*"recognit" + 0.049*"serverless"')
(6, '0.107*"org" + 0.102*"content" + 0.070*"http" + 0.051*"super" + 0.047*"select"')
(7, '0.267*"build" + 0.122*"websit" + 0.049*"card" + 0.043*"use" + 0.040*"option"')
(8, '0.345*"gener" + 0.080*"singl" + 0.067*"configur" + 0.050*"need" + 0.045*"adversari"')
(9, '0.199*"file" + 0.143*"plugin" + 0.110*"video" + 0.079*"json" + 0.061*"deploy"')
(10, '0.132*"ai" + 0.096*"rest" + 0.049*"benchmark" + 0.048*"news" + 0.040*"detector"')
(11, '0.132*"2018" + 0.062*"gan" + 0.058*"hash" + 0.043*

In [96]:
# SKLEARN LEARN LDA
from __future__ import print_function
from time import time
from random import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 7000
n_features = 100
n_components = 10
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [91]:
# shuffle the data
en_shuffle = en_text_list
shuffle(en_shuffle)
print(en_shuffle)
len(en_shuffle)

['百思不得姐4.5.6版本高仿', 'Symfony Maker Bundle', 'Seamless copy-and-paste of images with Poisson Blending.', 'Simple plotting tool for the terminal', 'Component testing utilities for Vue.js', ':fire: CHAOS allow generate payloads and control remote Windows systems.', '√A customized LayoutManager - fade and shrink the head itemView when scrolling.', "PyTorch reimplementation of Google's TensorFlow CNNs for keyword spotting", '🚎 - Tiny message bus', '照片/图片选择器 - 支持LivePhoto、GIF图片选择、3DTouch预览、在线下载iCloud上的资源、浏览网络图片 功能    Imitation weibo photo/image picker - support for LivePhoto, GIF image selection, 3DTouch preview, Download the resources on iCloud online, browse the web image function', 'Implementations of different VAE-based semi-supervised and generative models in PyTorch', 'Ever hid debug functions in your UI? Here is now a clean way to do it!', 'Torch implementation of "Enhanced Deep Residual Networks for Single Image Super-Resolution"', 'Docker Compose UI: Lifeboat is an easy way to launch

11143

In [92]:
# Get data sample
en_samples = en_shuffle[:n_samples]
len(en_samples)

7000

In [93]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf-idf features for NMF...
done in 0.145s.
Extracting tf features for LDA...
done in 0.125s.



In [94]:
tf.shape
print(tf)

  (3, 89)	1
  (3, 82)	1
  (4, 48)	1
  (4, 95)	1
  (4, 19)	1
  (5, 98)	1
  (7, 86)	1
  (7, 38)	1
  (7, 76)	1
  (9, 97)	1
  (9, 79)	1
  (9, 84)	1
  (9, 43)	3
  (10, 9)	1
  (10, 76)	1
  (11, 96)	1
  (11, 91)	1
  (12, 65)	1
  (12, 26)	1
  (12, 44)	1
  (12, 43)	1
  (13, 74)	1
  (13, 30)	1
  (13, 96)	1
  (13, 91)	1
  :	:
  (6991, 65)	1
  (6992, 46)	1
  (6992, 83)	1
  (6992, 68)	1
  (6992, 44)	1
  (6993, 63)	1
  (6993, 77)	1
  (6993, 9)	1
  (6993, 43)	1
  (6995, 52)	1
  (6995, 59)	1
  (6995, 15)	1
  (6996, 69)	1
  (6996, 93)	1
  (6997, 50)	1
  (6997, 71)	1
  (6997, 38)	1
  (6998, 47)	1
  (6998, 53)	1
  (6998, 64)	1
  (6998, 66)	1
  (6999, 39)	1
  (6999, 25)	1
  (6999, 5)	1
  (6999, 70)	1


In [97]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=7000 and n_features=100...
done in 0.271s.

Topics in NMF model (Frobenius norm):
Topic #0: tool command line source linux open files network data security kubernetes build file applications http windows performance ui building projects
Topic #1: code source 2017 https open javascript github com language framework swift text model time files ui repository network detection extension
Topic #2: react native app components component ios apps ui built build like create web api css design use com material example
Topic #3: android app ios view use architecture apps easy project client kotlin material design way plugin native like github file java
Topic #4: library javascript data text ui line swift lightweight server source fast support platform built high open file building like kotlin
Topic #5: learning deep implementation machine tensorflow pytorch networks neural image resources network model 2017 list project curate